In [ ]:
#hide
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
#hide
%load_ext autoreload 
%autoreload 2
%cd "/content/drive/MyDrive/Coding/ModelAssistedLabel"

/content/drive/MyDrive/Coding/ModelAssistedLabel


In [ ]:
#hide
# %run '_Synch.ipynb' 

# Model-asisted Labeling with YOLOv5
> bootstrapping image annotation

# Background
Object detection is great! ... if your labeled dataset already exists. I wanted to use machine learning to turn my regular rowing machine into a "smart" rowing machine (specifically: I want to track my workout stats).

Unfortunately, I was unable to find a suitable existing set of labeled LCD digits.

After working through [a Roboflow tutorial]( https://models.roboflow.com/object-detection/yolov5), I started to use Roboflow to annotate and store my images. 

I hated annotating my images by hand. Once my model began making reasonable guesses, I resolved to enlist the model's help in labeling new images. (I ended up building a [key-driven image labeler](https://github.com/PhilBrockman/autobbox) to modify my model's predictions, but that codebase is no longer being maintained.)

I found the files responsible for training and employing models and wrote wrappers around the relevant calls

## Expected Inputs:
* Either:
  - **weight** file OR
  - **labeled images**
      + All of the images and labels must be in a common folder (subfolders allowed).
      + labels must be in [YOLOv5 format](https://github.com/AlexeyAB/Yolo_mark/issues/60).
* And:
  - **unlabeled images**


> Note: Image/label pairs are based on their base filename. For example `image.jpg/image.txt` would be paired as would `other_image5.jpg/other_image5.txt`.


## Expected Output:

* ***ZIP file*** that contains: 
    - `images/`
      + a copy of every image in **Unlabeled Data**
    - `labels/` (folder
      + result of running object detection on each image
    - a results folder produced by Ultralytic's `train.py` on the **Labeled Data**
    - `classmap.yaml` to preserve the identity of the classes


# Preparing Repository

Start by cloning https://github.com/ultralytics/yolov5.

In [ ]:
from ModelAssistedLabel.config import Defaults
import os

# enter root directory
os.chdir(Defaults().root)

# clone yolov5 repo and install requirements
# ensure GPU is enabled
Defaults.prepare_YOLOv5()

Setup complete. Using torch 1.8.0+cu101 _CudaDeviceProperties(name='Tesla P100-PCIE-16GB', major=6, minor=0, total_memory=16280MB, multi_processor_count=56)


# Building models from 

To make a model to annotate images, we first need annotated images. When I was first starting, I used Roboflow's tools to both annotate my images and to keep my data organized.

Once all of the labels/images are in a common folder called `repo`, we're ready to go:

In [ ]:
repo = "./Image Repo/labeled/Final Roboflow Export (841)"
name = "index model"

In [ ]:
from ModelAssistedLabel.train import AutoWeights

In [ ]:
wm = AutoWeights(repo, name, MAX_SIZE=None)

In [ ]:
%%time
wm.generate_weights(2000)

In [ ]:
wm.last_results_path

'./fromIndex-876096'

In [ ]:
!ls "test/images"

In [ ]:
runs = {
    './fromIndex-876096': {
        'description': "841 (?) "
    }
}